In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, uniform
import pandas as pd
import matplotlib.image as mpimg
from scipy.misc import derivative
from scipy.integrate import quad

### In this case we'll explore the case for multiple data set. We consider that the total distance is given by 
$$

d_{\rm tot} = \sum_{i} \omega_i d_{i} \left(X_{\rm obs,i}, X_{\rm sim,i} \right)
$$
### where $i$ runs over the number of multiple data set. 
### In this example we'll use SN Ia and BAO data to constrain CPL model.



In [ ]:
def total_distance(sn_obs, sn_err, sn_sim,
                     bao_obs, bao_err, bao_sim,
                     weights=None):
    if weights is None:
        weights = {'hubble': 1.0, 'sn': 1.0, 'bao': 1.0}

    d_sn = sn_distance(sn_obs, sn_err, sn_sim)
    d_bao = bao_distance(bao_obs, bao_err, bao_sim)
    
    # Weighted combination
    total_distance = (weights['sn'] * d_sn +
                     weights['bao'] * d_bao)
    
    return total_distance


def sn_distance(observed, errors, simulated):
    """Distance metric for Supernovae data (distance modulus)"""
    residuals = (observed - simulated) / errors
    return np.sum(residuals**2)

def bao_distance(observed, errors, simulated):
    """Distance metric for BAO data (distance ratios)"""
    residuals = (observed - simulated) / errors
    return np.sum(residuals**2)

In [5]:
Omega_m0 = 0.3
Omega_L0 = 0.7
h = 0.71
zvals = np.linspace(0,3,100)

In [ ]:
# cosmological distances 

def hubble_normalized_cpl(z, w0, wa):
    
    matter_term = Omega_m0 * (1 + z)**3
    
    de_term = Omega_L0 * (1+z)**(3*(1+w0+wa)) * np.exp(-3*wa*z/(1+z))
    return  H0 * np.sqrt(matter_term + de_term)


def H():
       return 100*h

def HUBz(z):
    return H()*hubble_normalized_cpl(z)

def inverse_hubble_normalized_z(z):

    return 1./hubble_normalized_cpl(z)

def hubble_normalized_a(a):
    return hubble_normalized_cpl(1./a-1)

def hubble_prime_normalized_a(a):
    return derivative(hubble_normalized_a, a, dx=1e-6)

def D_H():
    """
    Hubble distance in units of MPc (David Hogg arxiv: astro-ph/9905116v4)
    """
    return 2997.98/h

def comoving_distance_z(z1):
    return D_H() *quad(inverse_hubble_normalized_z,0, z1,epsabs=1e-6, epsrel=1e-6, limit=200)[0]

# angular diameter distance in units of MPc
def angular_diameter_distance_z(z):
    """
    Angular diameter distance as function of redshift z
    in units of MPc
    """
    d_c = comoving_distance_z(z)/(1.+z)
    return d_c

# luminosity distance in units of MPc
def luminosity_distance_z(self, z):
    d_c = comoving_distance_z(z)
    return (1 + z) * d_c

